## 난임 환자 대상 임신 성공 여부 예측

### LGAimers 6th 온라인 해커톤

Import

In [13]:
import pandas as pd
import lightgbm as lgb
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
import lightgbm as lgb
import xgboost as xgb
from catboost import CatBoostClassifier
from sklearn.ensemble import RandomForestClassifier

import numpy as np
import pandas as pd
import lightgbm as lgb
from sklearn.model_selection import KFold, train_test_split
from sklearn.preprocessing import OrdinalEncoder
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, confusion_matrix
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

In [14]:
import numpy as np
import pandas as pd
import lightgbm as lgb
from sklearn.cluster import KMeans
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
from sklearn.preprocessing import StandardScaler

### Data Load

In [15]:
# 데이터 로드
IVF_train = pd.read_csv('../data/IVF_train_dataset_20.csv')
IVF_test = pd.read_csv('../data/IVF_test_dataset_20.csv')

DI_train = pd.read_csv('../data/DI_train_dataset_20.csv')
DI_test = pd.read_csv('../data/DI_test_dataset_20.csv')

In [16]:
# ID 열을 제외한 특성과 타겟 변수 분리
IVF_X = IVF_train.drop(['임신_성공_여부', 'ID'], axis=1)
IVF_y = IVF_train['임신_성공_여부']

DI_X = DI_train.drop(['임신_성공_여부', 'ID'], axis=1)
DI_y = DI_train['임신_성공_여부']

### 인코딩 

In [17]:
IVF_categorical_columns = [
    "시술_시기_코드",
    "시술_당시_나이",
    "임신_시도_또는_마지막_임신_경과_연수",
    "배란_유도_유형",
    "배아_생성_주요_이유",
    "난자_출처",
    "정자_출처",
    "난자_기증자_나이",
    "정자_기증자_나이",
    "변환된_특정_시술_유형",
    "채취_해동_차이",
    "해동_혼합_차이",
    "혼합_이식_차이",
    "이식_해동_차이"
]

In [18]:
DI_categorical_columns = [
    "시술_시기_코드",
    "시술_당시_나이",
    "임신_시도_또는_마지막_임신_경과_연수",
    "정자_기증자_나이",
    "변환된_특정_시술_유형"
]

In [19]:
# 모든 범주형 변수를 문자열로 변환
IVF_X[IVF_categorical_columns] = IVF_X[IVF_categorical_columns].astype(str)
DI_X[DI_categorical_columns] = DI_X[DI_categorical_columns].astype(str)
IVF_test[IVF_categorical_columns] = IVF_test[IVF_categorical_columns].astype(str)
DI_test[DI_categorical_columns] = DI_test[DI_categorical_columns].astype(str)

# OrdinalEncoder를 사용하여 범주형 변수 인코딩
IVF_encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)
DI_encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)

IVF_X[IVF_categorical_columns] = IVF_encoder.fit_transform(IVF_X[IVF_categorical_columns])
DI_X[DI_categorical_columns] = DI_encoder.fit_transform(DI_X[DI_categorical_columns])
IVF_test[IVF_categorical_columns] = IVF_encoder.transform(IVF_test[IVF_categorical_columns])
DI_test[DI_categorical_columns] = DI_encoder.transform(DI_test[DI_categorical_columns])

In [20]:
# 데이터 분할
IVF_X_train, IVF_X_test, IVF_y_train, IVF_y_test = train_test_split(IVF_X, IVF_y, test_size=0.2, random_state=42)
DI_X_train, DI_X_test, DI_y_train, DI_y_test = train_test_split(DI_X, DI_y, test_size=0.2, random_state=42)

## Modeling

train 데이터의 일부를 검증데이터로 사용하여 성능지표 출력

In [21]:
# K-Fold를 사용하여 IVF와 DI의 잘못된 예측 데이터 수집
kf = KFold(n_splits=5, shuffle=True, random_state=42)

IVF_wrong_predictions = []
DI_wrong_predictions = []

# K-Fold 교차 검증을 통한 잘못된 예측 수집 함수
def collect_wrong_predictions(X, y, wrong_predictions, model_params):
    for train_idx, val_idx in kf.split(X):
        X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
        y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]
        
        model = lgb.LGBMClassifier(**model_params)
        model.fit(X_train, y_train)
        y_pred = model.predict(X_val)
        
        wrong_idx = val_idx[y_pred != y_val.values]
        wrong_predictions.append(X.iloc[wrong_idx])

# IVF와 DI에 대해 잘못된 예측 수집
IVF_model_params = {
    'n_estimators': 4471,
    'num_leaves': 13,
    'max_depth': 279,
    'learning_rate': 0.007075124517450591,
    'min_child_samples': 26,
    'subsample': 0.29772991936701476,
    'colsample_bytree': 0.8913054521763838,
    'reg_alpha': 0.0004860363321690653,
    'reg_lambda': 311.08056657247363,
    'min_split_gain': 0.18214905183450955,
    'random_state': 42,
    'boosting_type': 'gbdt',
    'verbose': -1
}

DI_model_params = {
    'n_estimators': 1816,
    'num_leaves': 3926,
    'max_depth': 259,
    'learning_rate': 0.00238377640011148,
    'min_child_samples': 1,
    'subsample': 0.7610056627240331,
    'colsample_bytree': 0.6655579164853634,
    'reg_alpha': 0.00025227758337188327,
    'reg_lambda': 76.744107215122684,
    'min_split_gain': 0.007773520329665474,
    'random_state': 42,
    'boosting_type': 'gbdt',
    'verbose': -1
}

collect_wrong_predictions(IVF_X, IVF_y, IVF_wrong_predictions, IVF_model_params)
collect_wrong_predictions(DI_X, DI_y, DI_wrong_predictions, DI_model_params)

# 잘못된 예측 데이터 결합
wrong_data = pd.concat(IVF_wrong_predictions + DI_wrong_predictions)

In [ ]:
# NaN 값을 0으로 대체
wrong_data = wrong_data.fillna(0)

# KMeans 군집화 모델 학습
kmeans = KMeans(n_clusters=2, random_state=42)
clusters = kmeans.fit_predict(wrong_data)

# 군집 결과를 데이터프레임에 추가
wrong_data['cluster'] = clusters

# 군집화된 데이터 확인
print(wrong_data.groupby('cluster').size())

cluster
0    14343
1    50708
dtype: int64


In [25]:
# 테스트 데이터에 대한 예측 수행
IVF_model_final = lgb.LGBMClassifier(**IVF_model_params)
IVF_model_final.fit(IVF_X, IVF_y)
IVF_test_pred = IVF_model_final.predict(IVF_test.drop('ID', axis=1))

DI_model_final = lgb.LGBMClassifier(**DI_model_params)
DI_model_final.fit(DI_X, DI_y)
DI_test_pred = DI_model_final.predict(DI_test.drop('ID', axis=1))

# 테스트 데이터 결합
all_test_data = pd.concat([IVF_test, DI_test])
all_test_pred = np.concatenate([IVF_test_pred, DI_test_pred])

# NaN 값을 0으로 대체
all_test_data = all_test_data.fillna(0)

# 테스트 데이터에 대한 군집화 예측
test_clusters = kmeans.predict(all_test_data.drop(['ID'], axis=1))

# 군집화 모델이 잘못된 예측을 할 것으로 예측한 행의 예측값 반전
all_test_pred[test_clusters == 1] = 1 - all_test_pred[test_clusters == 1]

# 최종 제출 파일 생성
IVF_submission = pd.DataFrame({'ID': IVF_test['ID'], 'probability': all_test_pred[:len(IVF_test)]})
DI_submission = pd.DataFrame({'ID': DI_test['ID'], 'probability': all_test_pred[len(IVF_test):]})

submission = pd.concat([IVF_submission, DI_submission])
submission.to_csv('../submission/code21_1_submit.csv', index=False)

print("제출 파일(code21_1_submit.csv) 생성 완료!")

제출 파일(code21_1_submit.csv) 생성 완료!


In [28]:
submission.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 90067 entries, 0 to 2175
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   ID           90067 non-null  object
 1   probability  90067 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 2.1+ MB


엑셀 제대로 뽑혔나 확인

In [30]:
import pandas as pd

# 두 CSV 파일을 읽어옵니다.
df1 = pd.read_csv('../submission/code21_1_submit.csv')
df2 = pd.read_csv('../submission/code21_submit.csv')

# '임신_성공_여부' 열을 비교합니다.
comparison = df1['probability'] == df2['probability']

# 차이점이 있는 행을 출력합니다.
differences = df1[~comparison]

len(differences)

66033

In [29]:
import pandas as pd

# 두 CSV 파일을 읽어옵니다.
df1 = pd.read_csv('../submission/code21_1_submit.csv')
df2 = pd.read_csv('../submission/code20_submit.csv')

# '임신_성공_여부' 열을 비교합니다.
comparison = df1['probability'] == df2['probability']

# 차이점이 있는 행을 출력합니다.
differences = df1[~comparison]

len(differences)

90067

.